In [ ]:
import config
from phemex import *
import json
import pandas as pd
from datetime import timedelta, datetime
import nest_asyncio
import time
import ta
import numpy as np
import ccxt
# import talib
# from talib import stream
from IPython.display import clear_output
from scipy.signal import argrelextrema
from collections import deque

nest_asyncio.apply()
import asyncio
import websockets
clients = {}
# Create an instance of the Phemex exchange
exchange = initExchange()
symbol = config.TRADE_SYMBOL
trades = pd.DataFrame(columns=['timestamp', 'side', 'priceEp', 'volume'])
tradesPrices = []
ohlcvData = pd.DataFrame(
    columns=['open', 'high', 'low', 'close', 'volume', 'timestamp'])
klineData = pd.DataFrame(
    columns=['timestamp', 'interval', 'lastclose', 'open', 'high', 'low', 'close', 'volume', 'turnoverEv'])
tick_size = 0.5
quantity = 100
stop_distance = 10
stop_price = None

# Set up exchange client
exchange = initExchange()

In [ ]:
class Int64Encoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.int64):
            return int(obj)
        return super().default(obj)

In [ ]:
prevTickers = None
wait = None
async def placeStoplossOrder(lastMbtcTick,lastClose,position,quantity,addStoploss=False):  
    global stop_price, prevTickers

    # Get the current time
    now = datetime.now()
    # Get the current second
    second = now.second

    stop_distance = 30 * 10000  
    
    try:
        existing_orders = exchange.fetch_open_orders(symbol=config.TRADE_SYMBOL)  
    except Exception as e:
        print('Error {}'.format(e))
        pass 
    
    limitFound = False 
    stoplossFound = False 
    stoploss_order = None      
    for order in existing_orders:
        if order['info']['orderType'] == 'Limit' and not limitFound:
            side = order['info']['side']
            limitFound = True  
        if order['info']['orderType'] == 'Stop' and not stoplossFound:
            side = order['info']['side']
            stoploss_order = order
            stoplossFound = True  
    # Check if .MBTC ticker is above last price
    # if int(lastMbtcTick) > int(lastClose):
    print(position['side'],stoplossFound,addStoploss)
    # Check if there is a short position
    if position['side'] == 'Sell' and stoplossFound == False and addStoploss == True: 
        
        # Check if the current second is 29 or 59
        # if second == 29 or second == 59:
        print("The current second is 29 or 59.")
        # Set stop loss slightly below liquidation price
        stop_price = int(position['liquidationPrice'] - stop_distance)
        params = {
                "orderType": "Stop",
                "timeInForce": "ImmediateOrCancel",
                "trigger": "ByMarkPrice",
                "stopDirection": "Falling",
                "stopPxEp": stop_price,
                "stopPx": stop_price * 0.0001,
                # "stopLossEp": stop_price,
                "execInst": "CloseOnTrigger",
        }         
        stoploss_order = exchange.create_order(symbol=config.TRADE_SYMBOL, type='stop', side='buy', price=position['liquidationPrice'] * 0.0001, amount=quantity, params=params)     
        stoplossFound = True                 
        # else:
        #     print("The current second is not 29 or 59.")     
        #     stoplossFound = False       

        # trailingOrder = exchange.create_order(symbol = config.TRADE_SYMBOL, type = 'Stop', side = 'buy', amount = quantity, price = lastClose+50, params = {
        #     'stopPrice': stop_price,
        #     'ordType': 'StopLimit',
        #     'pegPriceType': 'TrailingStopPeg',
        #     'pegOffsetValueEp': 0, #10000, # needs to be scaled
        #     'triggerType': 'ByMarkPrice'
        #     }
        # )                  

    # Update the stop-loss and limit orders
    elif position['side'] == 'Buy' and stoplossFound == True and addStoploss == False:
        # Remove stop loss
        exchange.cancel_order(stoploss_order['id'], symbol)                  
        cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
        stoploss_order = None                                             
# Check if .MBTC ticker is below last price
# elif int(lastMbtcTick) < int(lastClose):
    # Check if there is a long position
    if position['side'] == 'Buy' and stoplossFound == False and addStoploss == True:
        # Set stop loss slightly above liquidation price
        # if second == 29 or second == 59:
        stop_price = int(position['liquidationPrice'] + stop_distance)
        params = {
                "orderType": "Stop",
                "timeInForce": "ImmediateOrCancel",
                "trigger": "ByMarkPrice",
                "stopDirection": "Rising",
                "stopPxEp": stop_price,
                "stopPx": stop_price * 0.0001,
                # "stopLossEp": stop_price,
                "execInst": "CloseOnTrigger",
        }
        price = position['liquidationPrice']
        stoploss_order = exchange.create_order(symbol=config.TRADE_SYMBOL, type='stop', side='sell', price=price, amount=quantity, params=params)      
        stoplossFound = True  

        # trailingOrder = exchange.create_order(symbol = config.TRADE_SYMBOL, type = 'Stop', side = 'sell', amount = quantity, price = lastClose-50, params = {
        #     'stopPrice': stop_price,
        #     'ordType': 'StopLimit',
        #     'pegPriceType': 'TrailingStopPeg',
        #     'pegOffsetValueEp': 0, #10000, # needs to be scaled
        #     'triggerType': 'ByMarkPrice'
        #     }
        # )         
        # else:
        #     print("The current second is not 29 or 59.")     
        #     stoplossFound = False                                               
    # Update the stop-loss and limit orders
    elif position['side'] == 'Sell' and stoplossFound == True  and addStoploss == False:
        # Remove stop loss
        exchange.cancel_order(stoploss_order['id'], symbol)                  
        cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
        stoploss_order = None                           

In [ ]:
prevStoplossSell = None
async def stochStrategySell(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity):
    global prevStoplossSell
    stoploss = None
    # check for stoploss conditions
    if latest_k > latest_d:
        # print(f'latest_k {latest_k} > latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 80 <= latest_k <= 100 and 50 <= latest_d < 80:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 50 <= latest_k < 80 and 80 <= latest_d < 100:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 50 <= latest_k < 80 and 20 <= latest_d < 50:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 20 <= latest_k < 50 and 0 <= latest_d < 20:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
    elif latest_k < latest_d:
        # print(f'latest_k {latest_k} < latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
        elif 50 <= latest_k < 80 and 80 <= latest_d <= 100:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 50 <= latest_k < 80 and 50 <= latest_d < 80:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 20 <= latest_k < 50 and 50 <= latest_d < 80:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 0 <= latest_k < 20 and 20 <= latest_d < 50:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
        await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
    else:
        print(f'latest_k {latest_k} == latest_d {latest_d}')
    
    if prevStoplossSell != None and prevStoplossSell != stoploss:
        print(f'Strategy: Short')
        # Check if .MBTC ticker is above last price
        if int(lastMbtcTick) > int(lastClose):
            print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')    
        # Check if .MBTC ticker is below last price
        elif int(lastMbtcTick) < int(lastClose):
            print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-') 
        else:
            print(f'-= MBTC {lastMbtcTick} equal Last close {lastClose} =-')                  
        print(f'latest_k: {latest_k} latest_d: {latest_d} stoploss: {stoploss}')
        prevStoplossSell = stoploss
    elif prevStoplossSell == None:
        print(f'Strategy: Short')
        # Check if .MBTC ticker is above last price
        if int(lastMbtcTick) > int(lastClose):
            print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')    
        # Check if .MBTC ticker is below last price
        elif int(lastMbtcTick) < int(lastClose):
            print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-')   
        else:
            print(f'-= MBTC {lastMbtcTick} equal Last close {lastClose} =-')                             
        print(f'latest_k: {latest_k} latest_d: {latest_d} stoploss: {stoploss}')
        prevStoplossSell = stoploss
    else:
        print(f'Strategy: Short')
        # Check if .MBTC ticker is above last price
        if int(lastMbtcTick) > int(lastClose):
            print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')    
        # Check if .MBTC ticker is below last price
        elif int(lastMbtcTick) < int(lastClose):
            print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-')  
        else:
            print(f'-= MBTC {lastMbtcTick} equal Last close {lastClose} =-')                     
        print(f'latest_k: {latest_k} latest_d: {latest_d} stoploss: {stoploss}')         

In [ ]:
prevStoplossBuy = None
async def stochStrategyBuy(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity):
    global prevStoplossBuy
    stoploss = None
    # check for stoploss conditions
    if latest_k > latest_d:
        # print(f'latest_k {latest_k} > latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 80 <= latest_k <= 100 and 50 <= latest_d < 80:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 50 <= latest_k < 80 and 50 <= latest_d < 80:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 50 <= latest_k < 80 and 20 <= latest_d < 50:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 20 <= latest_k < 50 and 0 <= latest_d < 20:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
    elif latest_k < latest_d:
        # print(f'latest_k {latest_k} < latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 50 <= latest_k < 80 and 80 <= latest_d <= 100:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 50 <= latest_k < 80 and 50 <= latest_d < 80:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 20 <= latest_k < 50 and 50 <= latest_d < 80:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 0 <= latest_k < 20 and 20 <= latest_d < 50:
            # remove stoploss
            # print("Removing stoploss...")
            stoploss = False
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # add stoploss
            # print("Adding stoploss...")
            stoploss = True
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=stoploss)
    else:
        print(f'latest_k {latest_k} == latest_d {latest_d}')

    if prevStoplossBuy != None and prevStoplossBuy != stoploss:
        print(f'Strategy: Long')
        # Check if .MBTC ticker is above last price
        if int(lastMbtcTick) > int(lastClose):
            print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')    
        # Check if .MBTC ticker is below last price
        elif int(lastMbtcTick) < int(lastClose):
            print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-')    
        else:
            print(f'-= MBTC {lastMbtcTick} equal Last close {lastClose} =-')                   
        print(f'latest_k: {latest_k} latest_d: {latest_d} stoploss: {stoploss}')
        prevStoplossBuy = stoploss
    elif prevStoplossBuy == None:
        print(f'Strategy: Long')
        # Check if .MBTC ticker is above last price
        if int(lastMbtcTick) > int(lastClose):
            print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')    
        # Check if .MBTC ticker is below last price
        elif int(lastMbtcTick) < int(lastClose):
            print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-')  
        else:
            print(f'-= MBTC {lastMbtcTick} equal Last close {lastClose} =-')                     
        print(f'latest_k: {latest_k} latest_d: {latest_d} stoploss: {stoploss}')
        prevStoplossBuy = stoploss  
    else:
        print(f'Strategy: Long')
        # Check if .MBTC ticker is above last price
        if int(lastMbtcTick) > int(lastClose):
            print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')    
        # Check if .MBTC ticker is below last price
        elif int(lastMbtcTick) < int(lastClose):
            print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-')  
        else:
            print(f'-= MBTC {lastMbtcTick} equal Last close {lastClose} =-')                     
        print(f'latest_k: {latest_k} latest_d: {latest_d} stoploss: {stoploss}')                  

In [ ]:
async def strategy(lastMbtcTick,lastClose,psar_indicator,lastRSO,latest_k,latest_d):
    global stop_price
  
    try:
        position = getPositionInfo()[0]
        quantity = position['amount']  
        difference = abs(int(lastMbtcTick) - int(lastClose))                                           
        if position['side'] == 'Sell':
            await stochStrategySell(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity)
        elif position['side'] == 'Buy':
            await stochStrategyBuy(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity)        
        # print(f'position: {position}')
    except Exception as e:
        print('Error {}'.format(e))
        pass
 

In [ ]:
def handle_kline(kline):
    # Sort kline data by timestamp in ascending order
    kline = sorted(kline, key=lambda x: x[0])
    # Print sorted kline data
    return kline

In [ ]:
def execute_trade(symbol, trend, klines):
    if trend == 'long':
        lower_lows = np.array([float(kline.low) for kline in klines])
        lower_lows_rolling_min = np.minimum.reduce(np.split(lower_lows, len(lower_lows) / 3))
        trade_price = lower_lows_rolling_min[0]
        order_response = exchange.place_order(symbol=symbol, side='buy', order_type='limit', price=trade_price, quantity=1)
    else:
        higher_highs = np.array([float(kline.high) for kline in klines])
        higher_highs_rolling_max = np.maximum.reduce(np.split(higher_highs, len(higher_highs) / 3))
        trade_price = higher_highs_rolling_max[0]
        order_response = exchange.place_order(symbol=symbol, side='sell', order_type='limit', price=trade_price, quantity=1)
    return order_response


In [ ]:
prevStoplossSell = None
async def strategySell(lastBid,lastTradePrice,lastTradeSide,lastPrice,LastMBTCPrice,latest_k,latest_d,position,quantity):
    global prevStoplossSell
    stoploss = None
    print(f'lastBid {lastBid} vs lastPrice {lastPrice}')
    # check for stoploss conditions
    if lastBid < lastPrice:
        # remove stoploss
        # print("Removing stoploss...")
        stoploss = False
        await placeStoplossOrder(LastMBTCPrice,lastPrice,position,quantity,addStoploss=stoploss)
    else:
        # add stoploss
        # print("Adding stoploss...")
        stoploss = True
        await placeStoplossOrder(LastMBTCPrice,lastPrice,position,quantity,addStoploss=stoploss)  

In [ ]:
prevStoplossBuy = None
async def strategyBuy(lastBid,lastTradePrice,lastTradeSide,lastPrice,LastMBTCPrice,latest_k,latest_d,position,quantity):
    global prevStoplossBuy
    stoploss = None
    print(f'lastBid {lastBid} vs lastPrice {lastPrice}')
    # check for stoploss conditions
    if lastBid < lastPrice:
        # remove stoploss
        # print("Removing stoploss...")
        stoploss = False
        await placeStoplossOrder(LastMBTCPrice,lastPrice,position,quantity,addStoploss=stoploss)
    else:
        # add stoploss
        # print("Adding stoploss...")
        stoploss = True
        await placeStoplossOrder(LastMBTCPrice,lastPrice,position,quantity,addStoploss=stoploss)      

In [ ]:
async def strategyStoploss(lastBid,lastTradePrice,lastTradeSide,lastPrice,LastMBTCPrice,latest_k,latest_d):
    global stop_price
    try:
        position = getPositionInfo()[0]
        quantity = position['amount']  
        difference = abs(int(LastMBTCPrice) - int(lastPrice))                                           
        if lastTradeSide == 'Sell':
            await strategySell(lastBid,lastTradePrice,lastTradeSide,lastPrice,LastMBTCPrice,latest_k,latest_d,position,quantity)
        elif lastTradeSide == 'Buy':
            await strategyBuy(lastBid,lastTradePrice,lastTradeSide,lastPrice,LastMBTCPrice,latest_k,latest_d,position,quantity)        
        # print(f'position: {position}')
        # time.sleep(20000)
    except Exception as e:
        print('Error {}'.format(e))
        pass

In [ ]:
async def resampleKlines(klineData, interval='1T'):
    resampled = klineData.resample(interval).last().dropna()
    resampled['time'] = (resampled.index - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    # resampled = resampled.reset_index()  # move 'timestamp' column back to DataFrame
    # resampled = resampled.set_index('timestamp')  # set 'timestamp' column as the index
    # print('resampled',resampled,interval)
    # time.sleep(5000)
    return resampled.sort_index()  # sort DataFrame by index


In [ ]:
async def getPrices(klineData):
    # print('getPrices')
    prices = []
    for index, row in klineData.iterrows():
        # print('row',row)
        prices.append({'time':row['timestamp'],'value':round(row['close'] * 0.0001,2),})
    return prices

In [ ]:
def getHigherHighs(data: np.array, timestamps: np.array, order=4, K=2):
    '''
    Finds consecutive higher highs in price pattern.
    Must not be exceeded within the number of periods indicated by the width 
    parameter for the value to be confirmed.
    K determines how many consecutive highs need to be higher.
    '''
    # Get highs
    high_idx = argrelextrema(data, np.greater, order=order)[0]
    highs = data[high_idx]
    # Ensure consecutive highs are higher than previous highs and have the same slope
    extrema = []
    dummy=[]
    prev_slope = None
    for i, idx in enumerate(high_idx):
        if i == 0:
            dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
            prev_slope = None
            continue
        if highs[i] >= highs[i-1]:
            if len(dummy) >= K:
                # Find the slope of the line
                slope = highs[i-1] - highs[i-K]
                # Find the intercept of the line
                intercept = highs[i-K] - slope * (K-1)
                # Add the extended line to the extrema
                last_timestamp = timestamps[high_idx[i-K]]
                last_value = (slope * K + intercept) / 10000
                extrema.append([
                  {'time': last_timestamp, 'value': last_value},
                  {'time': timestamps[high_idx[i-K]], 'value': highs[i-K]/10000}
                ])
            dummy.clear()
            prev_slope = None
        else:
            if prev_slope is None:
                prev_slope = highs[i] - highs[i-1]
            elif highs[i] - highs[i-1] == prev_slope:
                pass
            else:
                dummy.clear()
                prev_slope = None

        dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
        if len(dummy) == K:
            extrema.append(dummy.copy())
            prev_slope = highs[i] - highs[i-K+1]
            dummy.pop(0)

    # Add the extended line to the last set of consecutive highs if it exists
    if len(dummy) >= K:
        # Find the slope of the line
        slope = highs[-1] - highs[-K]
        # Find the intercept of the line
        intercept = highs[-K] - slope * (K-1)
        # Add the extended line to the extrema
        last_timestamp = timestamps[high_idx[-K]]
        last_value = (slope * K + intercept) / 10000
        extrema.append([
          {'time': last_timestamp, 'value': last_value},
          {'time': timestamps[high_idx[-K]], 'value': highs[-K]/10000}
        ])

    return extrema


def getHigherLows(data: np.array, timestamps: np.array, order=4, K=2):
    '''
    Finds consecutive higher lows in price pattern.
    Must not be exceeded within the number of periods indicated by the width 
    parameter for the value to be confirmed.
    K determines how many consecutive lows need to be higher.
    '''
    # Get lows
    low_idx = argrelextrema(data, np.less, order=order)[0]
    lows = data[low_idx]
    # Ensure consecutive lows are higher than previous lows and have the same slope
    extrema = []
    dummy=[]
    prev_slope = None
    for i, idx in enumerate(low_idx):
        if i == 0:
            dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
            prev_slope = None
            continue
        if lows[i] <= lows[i-1]:
            if len(dummy) >= K:
                # Find the slope of the line
                slope = lows[i-1] - lows[i-K]
                # Find the intercept of the line
                intercept = lows[i-K] - slope * (K-1)
                # Add the extended line to the extrema
                last_timestamp = timestamps[low_idx[i-K]]
                last_value = (slope * K + intercept) / 10000
                extrema.append([
                  {'time': last_timestamp, 'value': lows[i-K]/10000},
                  {'time': timestamps[low_idx[i-K]], 'value': last_value}
                ])
            dummy.clear()
            prev_slope = None
        else:
            if prev_slope is None:
                prev_slope = lows[i] - lows[i-1]
            elif lows[i] - lows[i-1] == prev_slope:
                pass
            else:
                dummy.clear()
                prev_slope = None

        dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
        if len(dummy) == K:
            extrema.append(dummy.copy())
            prev_slope = lows[i] - lows[i-K+1]
            dummy.pop(0)
  
    # Add the extended line to the last set of consecutive lows if it exists
    if len(dummy) >= K:
        # Find the slope of the line
        slope = lows[-1] - lows[-K]
        # Find the intercept of the line
        intercept = lows[-K] - slope * (K-1)
        # Add the extended line to the extrema
        last_timestamp = timestamps[low_idx[-K]]
        last_value = (slope * K + intercept) / 10000
        extrema.append([
          {'time': last_timestamp, 'value': lows[-K]/10000},
          {'time': timestamps[low_idx[-K]], 'value': last_value}
        ])

    return extrema

def getLowerHighs(data: np.array, timestamps: np.array, order=4, K=2):
    '''
    Finds consecutive lower highs in price pattern.
    Must not be exceeded within the number of periods indicated by the width 
    parameter for the value to be confirmed.
    K determines how many consecutive highs need to be lower.
    '''
    # Get highs
    high_idx = argrelextrema(data, np.greater, order=order)[0]
    highs = data[high_idx]
    # Ensure consecutive highs are lower than previous highs and have the same slope
    extrema = []
    dummy = []
    prev_slope = None
    for i, idx in enumerate(high_idx):
        if i == 0:
            dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
            prev_slope = None
            continue
        if highs[i] <= highs[i-1]:
            if len(dummy) >= K:
                # Find the slope of the line
                slope = highs[i-K] - highs[i-1]
                # Find the intercept of the line
                intercept = highs[i-K] - slope * (K-1)
                # Add the extended line to the extrema
                last_timestamp = timestamps[high_idx[i-K]]
                last_value = (slope * K + intercept) / 10000
                extrema.append([
                  {'time': last_timestamp, 'value': last_value},
                  {'time': timestamps[high_idx[i-K]], 'value': highs[i-K]/10000}
                ])
            dummy.clear()
            prev_slope = None
        else:
            if prev_slope is None:
                prev_slope = highs[i-1] - highs[i]
            elif highs[i-1] - highs[i] == prev_slope:
                pass
            else:
                dummy.clear()
                prev_slope = None

        dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
        if len(dummy) == K:
            extrema.append(dummy.copy())
            prev_slope = highs[i-K+1] - highs[i]
            dummy.pop(0)
  
    # Add the extended line to the last set of consecutive highs if it exists
    if len(dummy) >= K:
        # Find the slope of the line
        slope = highs[-K] - highs[-1]
        # Find the intercept of the line
        intercept = highs[-K] - slope * (K-1)
        # Add the extended line to the extrema
        last_timestamp = timestamps[high_idx[-K]]
        last_value = (slope * K + intercept) / 10000
        extrema.append([
          {'time': last_timestamp, 'value': last_value},
          {'time': timestamps[high_idx[-K]], 'value': highs[-K]/10000}
        ])

    return extrema



def getLowerLows(data: np.array, timestamps: np.array, order=4, K=2):
    '''
    Finds consecutive lower lows in price pattern.
    Must not be exceeded within the number of periods indicated by the width 
    parameter for the value to be confirmed.
    K determines how many consecutive lows need to be lower.
    '''
    # Get lows
    low_idx = argrelextrema(data, np.less, order=order)[0]
    lows = data[low_idx]
    # Ensure consecutive lows are lower than previous lows and have the same slope
    extrema = []
    dummy=[]
    prev_slope = None
    for i, idx in enumerate(low_idx):
        if i == 0:
            dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
            prev_slope = None
            continue
        if lows[i] >= lows[i-1]:
            if len(dummy) >= K:
                # Find the slope of the line
                slope = lows[i-K] - lows[i-1]
                # Find the intercept of the line
                intercept = lows[i-K] - slope * (K-1)
                # Add the extended line to the extrema
                last_timestamp = timestamps[low_idx[i-K]]
                last_value = (slope * K + intercept) / 10000
                extrema.append([
                  {'time': last_timestamp, 'value': last_value},
                  {'time': timestamps[low_idx[i-K]], 'value': lows[i-K]/10000}
                ])
            dummy.clear()
            prev_slope = None
        else:
            if prev_slope is None:
                prev_slope = lows[i-1] - lows[i]
            elif lows[i-1] - lows[i] == prev_slope:
                pass
            else:
                dummy.clear()
                prev_slope = None

        dummy.append({'time':timestamps[idx],'value':data[idx]/10000})
        if len(dummy) == K:
            extrema.append(dummy.copy())
            prev_slope = lows[i-K+1] - lows[i]
            dummy.pop(0)
  
    # Add the extended line to the last set of consecutive lows if it exists
    if len(dummy) >= K:
        # Find the slope of the line
        slope = lows[-K] - lows[-1]
        # Find the intercept of the line
        intercept = lows[-K] - slope * (K-1)
        # Add the extended line to the extrema
        last_timestamp = timestamps[low_idx[-K]]
        last_value = (slope * K + intercept) / 10000
        extrema.append([
          {'time': last_timestamp, 'value': last_value},
          {'time': timestamps[low_idx[-K]], 'value': lows[-K]/10000}
        ])

    return extrema


In [ ]:
async def returnCurrentEntry(currentPrice,lastHH,lastHL,lastLH,lastLL):
    positions = []
    # Check for long entry
    if currentPrice > lastHH and currentPrice > lastHL:
        print("Long entry signal")
        positions.append({
            'entryPrice': currentPrice,
            'stopLoss': lastHL,
            'takeProfit': lastHH,
            'type': 'long',
        })

    # Check for short entry
    if currentPrice < lastLL and currentPrice < lastLH:
        print("Short entry signal")
        positions.append({
            'entryPrice': currentPrice,
            'stopLoss': lastLL,
            'takeProfit': lastLH,
            'type': 'short',
        })
        
    return positions

In [ ]:
klines = []
ticks = []
grouped_data = {}
lastTradePrice = None
lastTradeSide = None
lastBid = None
lastAsk = None
hh = []
hl = []
ll = []
lh = []
# prices = []

async def send_message(message):

    global ohlcvData, tradesPrices, klineData, klines,lastTradePrice,lastTradeSide,lastBid,lastAsk,higher_highs, higher_lows, lower_lows,lower_highs,prices,highestHighs
    type = None
    if message != None:
        # print(message)
        if 'trades' in message:
            lastTrade = json.loads(message)['trades'][-1]
            # print(f'lastTrade: {lastTrade}')
            lastTradeSide = lastTrade[1]
            lastTradePrice = lastTrade[2]
            
            # print(f'lastTradePrice: {lastTradePrice} lastTradeSide: {lastTradeSide}')            
            # tradesPrices = await getTradePrices(message,'1S')
            # lastTrade = tradesPrices.tail(1)
            # lastTradePrice = lastTrade['priceEp']
            # lastTradeSide = lastTrade['side'][0]
            # print(f'lastTradePrice: {lastTradePrice} lastTradeSide: {lastTradeSide}')
            # time.sleep(20000)

        if 'kline' in message:
            type = json.loads(message)['type']
            for kline in handle_kline(json.loads(message)['kline']):
                klines.append(kline)
            
            # prices = [{'time':pd.to_datetime(entry[0], unit='s'),'value':round(entry[6] * 0.0001,1)} for entry in klines] 

        if 'book' in message:
            # Get order book data for trading pair
            orderbook = json.loads(message)['book']
            bids = orderbook['bids']
            asks = orderbook['asks']

            # Calculate bid-ask spread
            if orderbook.get('bids') and orderbook.get('asks'):
                bid_price = round(float(bids[0][0] * 0.0001),1)
                ask_price = round(float(asks[0][0] * 0.0001),1)
                spread = round(((ask_price - bid_price) / ask_price)*100,1)

                # Determine current signal and price level for buy and sell walls
                if spread > 0.01:
                    signal = 1  # Long signal
                    sell_wall_price = round(asks[1][0] * 0.0001,1) if len(asks) > 1 else None
                elif spread < -0.01:
                    signal = -1  # Short signal
                    buy_wall_price = round(bids[1][0] * 0.0001,1) if len(bids) > 1 else None
                else:
                    signal = 0  # Neutral signal

                # Check for buy and sell walls
                buy_wall_size = 0
                buy_wall_price = 0
                for bid in bids:
                    if bid[1] > buy_wall_size:
                        buy_wall_size = bid[1]
                        buy_wall_price = round(float(bid[0] * 0.0001),1)

                sell_wall_size = 0
                sell_wall_price = 0
                for ask in asks:
                    if ask[1] > sell_wall_size:
                        sell_wall_size = ask[1]
                        sell_wall_price = round(float(ask[0] * 0.0001),1)

                # Adjust leverage based on signal
                if signal == 1:
                    leverage = 20
                elif signal == -1:
                    leverage = 20
                else:
                    leverage = 10

                # Set leverage for trading pair
                # exchange.set_leverage(leverage, config.TRADE_SYMBOL)
                # print(f'sell_wall_price {sell_wall_price} sell_wall_size {sell_wall_size} buy_wall_price {buy_wall_price} buy_wall_size {buy_wall_size}')
                # Place orders and manage position with leverage based on signal
                # if signal == 1:
                #     # Place long orders and manage position
                #     # ...
                #     print(f'Place long {signal} leverage {leverage} spread {spread}')
                # elif signal == -1:
                #     # Place short orders and manage position
                #     # ...
                #     print(f'Place Short {signal} leverage {leverage} spread {spread}')
                # else:
                #     # Close all positions
                #     # ...   
                #     print(f'Close all positions {signal} leverage {leverage} spread {spread}')  
                # time.sleep(20000)      

        if 'tick' in message:
            ticks.append(json.loads(message)['tick'])  
            if len(klines) > 30:   
                klineData = pd.DataFrame(klines,columns=['timestamp', 'interval', 'lastclose', 'open', 'high', 'low', 'close', 'volume', 'turnoverEv'])
                klineData['timestamp'] = pd.to_datetime(klineData['timestamp'], unit='s')
                klineData.set_index('timestamp', inplace=True)
                klineData = klineData.drop(columns=['interval', 'lastclose','turnoverEv'],axis=1)
                # print('Last kline',klineData.iloc[-1])
                # time.sleep(2000)
                
                # print(f'higher_highs {higher_highs}, higher_lows {higher_lows}, lower_lows {lower_lows},lower_highs {lower_highs}')
                # print(f'higher_highs {higher_highs}')
                # time.sleep(2000)
                # LastMBTCPrice = float(ticks[-1]['last'] * 0.001) 
                # lastPrice = float(klines[-1][6] * 0.001)
                # trend = check_high_low_trend(klineData)
                # print(f'trend seconds: {trend}')
                # slope = await getSlopeHighestHigh(klines)
                # print(f'slope: {slope}')
   
                # time.sleep(2000)           
                # # Filter klineData to only include rows with a new minute timestamp
                
                # newMinuteMask = klineData['timestamp'].diff().dt.seconds.gt(0)
                # klineData = klineData.loc[newMinuteMask]
                # Filter klineData to only include rows with 7-minute, 1-hour, or 1-day intervals
                # klineData['timestamp'] = pd.to_datetime(klineData['timestamp'], unit='ms')
                
                # klineData_1min = klineData.resample('1T').last().dropna()
                # trend = check_high_low_trend(klineData_1min)
                # # print(f'trend 1 minute: {trend}')
                # klineData_7min = klineData.resample('7T').last().dropna()
                # trend = check_high_low_trend(klineData_7min)
                # # print(f'trend 7 minute: {trend}')
                # klineData_15min = klineData.resample('15T').last().dropna()
                # trend = check_high_low_trend(klineData_15min)
                # # print(f'trend 15 minute: {trend}')
                # klineData_1h = klineData.resample('1H').last().dropna()
                # trend = check_high_low_trend(klineData_1h)
                # print(f'trend 1 hour: {trend}')
                # klineData_1d = klineData.resample('1D').last().dropna()
                # trend = check_high_low_trend(klineData_1d)
                # print(f'trend 1 day: {trend}')
                # klineData = pd.concat([klineData_1min, klineData_7min, klineData_1h, klineData_1d])
                # klineData.reset_index(inplace=True)

                # Calculate the Parabolic SAR
                # psar_indicator = ta.trend.PSARIndicator(high=klineData['high'], low=klineData['low'], close=klineData['close'])
                # klineData['sar'] = psar_indicator.psar()
                # lastSar = int(klineData.iloc[-1]['sar'] * 0.0001)


                # sma = talib.SMA(klineData['close'], timeperiod=14)
                # latest = stream.SMA(klineData['close'], timeperiod=14)
                # assert (sma.iloc[-1] - latest) < 0.00001

                # print(sma.iloc[-1], latest)#1.6180066666666686 1.6180066666666668
                # display(klineData['timestamp'].head(14))
                # display(klineData['timestamp'].tail(14))
                # fastk, fastd = talib.STOCHRSI(klineData['close'], timeperiod=14, fastk_period=14, fastd_period=3, fastd_matype=3)
                # # f, fd = stream.STOCHRSI(klineData['close'], timeperiod=14, fastk_period=14, fastd_period=3, fastd_matype=3)
                # # print(fastk.iloc[-1], f)
                # # print(fastd.iloc[-1], fd)

                # latest_k = fastk.iloc[-1]
                # latest_d = fastd.iloc[-1]
                # print(f'latest_k {latest_k} latest_d {latest_d}')
                # # print(f'Stream latest_k {f} latest_d {fd}')                
                # # assert (fastk.iloc[-1] - f) < 5#64.32089013974793 59.52628987038199

                # # print(fastk.iloc[-1], f)
                # # print(fastd.iloc[-1], fd)

                # print(f'Assert latest_k {latest_k} latest_d {latest_d}')
                # # print(f'Stream latest_k {f} latest_d {fd}')
                # time.sleep(10)
                

                # Calculate the Stochastic RSI values
                # k_period = 14
                # d_period = 3
                # rsi_period = 14
                # k_values = klineData['close'].tail(k_period)
                # rsi_values = klineData['close'].tail(rsi_period)
                # # Calculate the Stochastic RSI indicator
                # # rsi = talib.RSI(k_values, timeperiod=rsi_period)
                # fastk, slowk, slowd = talib.STOCHRSI(klineData['close'], k_period, k_period, d_period, d_period)
                # print(f'fastk {fastk}, slowk {slowk}, slowd {slowd}')
                # latest_k = fastk[-1]
                # latest_d = slowd[-1]
                # fastk_period = 14
                # slowk_matype = 0
                # slowk_period = 3
                # slowd_period = 3
                # slowd_matype = 0
                # rsi1 = talib.RSI(klineData['close'], rsi_period)
                # # print(rsi1)
                # k,d = talib.STOCH(rsi1, rsi1, rsi1, fastk_period=fastk_period,slowk_matype=slowk_matype,slowk_period=slowk_period,slowd_period=slowd_period,slowd_matype=slowd_matype)
                # # print('k-1',k.iloc[-1])
                # # print('d-1',d.iloc[-1])
                # d = talib.SMA(k, slowd_period)
                # # print('sma d-1',d.iloc[-1])
                # # Calculate the RSO Stochastic
                # # rso_indicator = ta.momentum.StochRSIIndicator(close=klineData['close'], window=14, smooth1=3, smooth2=3)
                # # klineData['rso_stoch_k'] = rso_indicator.stochrsi_k()
                # # klineData['rso_stoch_d'] = rso_indicator.stochrsi_d()
                # latest_k = k.iloc[-1]
                # latest_d = d.iloc[-1]  
                # # print(f'latest_k {latest_k} latest_d {latest_d}')            
                # # klineData['rso_stoch'] = rso_indicator.stochrsi()
                # # lastRSO = float(klineData.iloc[-1]['rso_stoch'] * 100)             
                # # print(f'lastMbtcTick: {lastMbtcTick} lastClose: {lastClose}')
                # # await strategy(lastMbtcTick,lastClose,lastSar,0,latest_k,latest_d)
                # if lastTradePrice > 0:
                #     await strategyStoploss(lastBid,lastTradePrice,lastTradeSide,lastPrice,LastMBTCPrice,latest_k,latest_d)
                # await mbtcStrategy(lastMbtcTick,secondLastMbtcTick,lastClose,secondLastClose)
        # if len(klines) > 100:
        #     await sarStrategy(klines)       
        for clientId in clients:
            client = clients[clientId]
            if len(klines) > 0:
                klinesResampled = await resampleKlines(klineData,interval=client['interval'])
                close = klinesResampled['close'].values
                timestamps = klinesResampled['time'].values
                order = 7
                K = 2

                hh = getHigherHighs(close,timestamps, order, K)
                hl = getHigherLows(close,timestamps, order, K)
                ll = getLowerLows(close,timestamps, order, K)
                lh = getLowerHighs(close,timestamps, order, K)
                currentPrice = close[-1]/10000
                lastHH = hh[-1][1]['value']
                lastHL = hl[-1][1]['value']
                lastLH = ll[-1][1]['value']
                lastLL = lh[-1][1]['value']
                print(f'lastHH {lastHH}')
                print(f'lastHL {lastHL}')
                print(f'lastLH {lastLH}')
                print(f'lastLL {lastLL}')
                # print(f'hh {hh}')
                # print(f'hl {hl}')
                # print(f'll {ll}')
                # print(f'lh {lh}')
                positions = await returnCurrentEntry(currentPrice,lastHH,lastHL,lastLH,lastLL)
                print('positions',positions)
                # time.sleep(200000)
                # highestHighs = await getHighestHighs(klinesResampled)
                # lowestHighs = await getLowestHighs(klinesResampled)
                # lowestLows = await getLowestLows(klinesResampled)
                # print('highestHighs',highestHighs)
                # print('Last kline',klines)
                # higher_highs, higher_lows, lower_lows,lower_highs = await findHighsLows(klinesResampled)
                sendMessage = json.dumps({'status':client['status'],'type':type,'positions':positions,'klines': klinesResampled.to_dict('records'),'ticks':ticks,'hh':hh, 'hl':hl, 'll':ll,'lh':lh},cls=Int64Encoder)
                if client['status'] == 'new':
                    await client['websocket'].send(sendMessage)
                    client['status'] = 'init'
                elif client['status'] == 'active':
                    await client['websocket'].send(sendMessage)

                
        clear_output(wait=True)

In [ ]:
# connect to Phemes websocket server
async def connectPhemexWS():
    global interval
    async for websocket in websockets.connect(config.WSSE_PHEMEX_TESTNET):
        try:
            print('Phemex Websocket Server is connected!')
            subscribe_ticker_msg = json.dumps({
                "id": 0,
                "method": "tick.subscribe",
                "params": [config.MBTC_SYMBOL]
            })
            await websocket.send(subscribe_ticker_msg)

            subscribe_kline_msg = json.dumps({
                "id": 0,
                "method": "kline.subscribe",
                "params": [config.TRADE_SYMBOL_BTCUSD,60]
            })
            await websocket.send(subscribe_kline_msg)
            
            subscribe_Trade_msg = json.dumps({
                "id": 0,
                "method": "trade.subscribe",
                "params": [config.TRADE_SYMBOL]
            })
            await websocket.send(subscribe_Trade_msg)  

            subscribe_orderbook_msg = json.dumps({
                "id": 0,
                "method": "orderbook.subscribe",
                "params": [config.TRADE_SYMBOL,True]
            })
            await websocket.send(subscribe_orderbook_msg) 

            while True:
                message = await websocket.recv()            
                await send_message(message)
        except websockets.ConnectionClosed:
            continue

In [ ]:
async def register(websocket):
    # Assign a unique ID to the client
    client_id = len(clients) + 1
    clients[client_id] = {"websocket":websocket,"status":"new","interval":'1T'}
    # await websocket.send(f"You are client {client_id}")
    print(f"Client {client_id} connected")
    return client_id

async def unregister(client_id):
    # Remove the client from the clients array
    del clients[client_id]
    print(f"Client {client_id} disconnected")

async def handle_message(client_id, message):
    # Handle incoming messages from the client
    print(f"Received message from client {client_id}: {json.loads(message)['status']}")
    # time.sleep(2000)
    # Store extra client information in clients array
    clients[client_id]["status"] = json.loads(message)['status']

async def handle_client(websocket, path):
    client_id = await register(websocket)
    try:
        async for message in websocket:
            await handle_message(client_id, message)
    except websockets.exceptions.ConnectionClosed:
        pass
    finally:
        # Unregister the client when they disconnect
        await unregister(client_id)

In [ ]:
# start python localhost websocket server
async def start_server():
    try:
        async with websockets.serve(handle_client, host="localhost", port=8764):
            print("Server started")
            await connectPhemexWS()
            await asyncio.Future()  # Keep the server running
    except Exception as e:
        print('Error {}'.format(e))
        pass

In [ ]:
if __name__ == '__main__':
    asyncio.run(start_server())